<a href="https://colab.research.google.com/github/pranavkantgaur/gamd_sr/blob/main/deliverible.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Objective
1. Demonstrate the existence and merits of algorithm alignment in GNNs for physical simulations, if any.

## Tasks
1. Loads a dataset from LJ, tip3w, tip4w, DFT
2. Optionally, generates MD dataset by passing a custom potential energy expressions as input
3. Loads relevant GNN model based on dataset complexity
4. Trains GNN and records edge messages of the converged model
5. Depending on the underlying custom potential, defines SR inputs and outputs
   1. Does edge message preprocessing to make sure input to output is a function
6. Trains SR
7. Plots pred vs gt curve for best SR equation
8. Discusses algorithm alignment of GNNs with the underlying physical phenomenon algorithm.
9. Perhaps demomstrates the benefits of algo. alignment:
   1. Whether algo. alignment is natural in GNNs or it happens becuase of the inductive biases?

### Load dataset

### Optionally, generate MD dataset

### Setup appropriate GNN model

### Train GNN and record edge messages

### Define SR inputs and outputs

### Train SR

### Plot pred vs gt curve for best SR equation

### Algorithm alignment of GNNs ?

### Benefits of algo. alignment ?